In [1]:
# importing essential libraries
import pandas as pd
import numpy as np
import os
import warnings
from pandasql import sqldf

In [2]:
# supressing warning messages
warnings.filterwarnings('ignore')

In [3]:
# locating the data
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_path = os.path.join(parent_dir,"data","teleco_users_data_original.csv")

#loading the data
df = pd.read_csv(data_path)
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311450e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082010e+14,3.366496e+10,3.552120e+13,9.16457E+15,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311450e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082020e+14,3.368185e+10,3.579400e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311450e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082000e+14,3.376063e+10,3.528150e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311450e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082010e+14,3.375034e+10,3.535660e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311450e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082010e+14,3.369980e+10,3.540700e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
# looking at available features
df.columns.to_list()

['Bearer Id',
 'Start',
 'Start ms',
 'End',
 'End ms',
 'Dur. (ms)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms).1',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Socia

In [5]:
# calculating percentages of missing values in each column 
def calculateMissing(df):
    df2 = df.isna().sum().to_frame().reset_index()
    df2.rename(columns = {'index':'variables', 0:'count'}, inplace = True)
    df2['percent'] = round(df2['count']*100/df.shape[0])
    print(df2)

calculateMissing(df)

                                   variables   count  percent
0                                  Bearer Id     991      1.0
1                                      Start       1      0.0
2                                   Start ms       1      0.0
3                                        End       1      0.0
4                                     End ms       1      0.0
5                                  Dur. (ms)       1      0.0
6                                       IMSI     570      0.0
7                              MSISDN/Number    1066      1.0
8                                       IMEI     572      0.0
9                         Last Location Name    1153      1.0
10                           Avg RTT DL (ms)   27829     19.0
11                           Avg RTT UL (ms)   27812     19.0
12                   Avg Bearer TP DL (kbps)       1      0.0
13                   Avg Bearer TP UL (kbps)       1      0.0
14               TCP DL Retrans. Vol (Bytes)   88146     59.0
15      

In [6]:
#droping columns with more than 30% missing values.
df3 = df.drop(df.columns[[14, 15, 24, 25, 31, 32, 33, 34, 35, 36]], axis=1) 
df3.shape

(150001, 45)

In [7]:
#checking data types of variables to decide how to handle missing values
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Bearer Id                       149010 non-null  float64
 1   Start                           150000 non-null  object 
 2   Start ms                        150000 non-null  float64
 3   End                             150000 non-null  object 
 4   End ms                          150000 non-null  float64
 5   Dur. (ms)                       150000 non-null  float64
 6   IMSI                            149431 non-null  float64
 7   MSISDN/Number                   148935 non-null  float64
 8   IMEI                            149429 non-null  float64
 9   Last Location Name              148848 non-null  object 
 10  Avg RTT DL (ms)                 122172 non-null  float64
 11  Avg RTT UL (ms)                 122189 non-null  float64
 12  Avg Bearer TP DL

In [8]:
#filling missing values of categorical variables with mode 
df3['Handset Type'] = df3['Handset Type'].fillna(df3['Handset Type'].mode()[0])
df3['Bearer Id'] = df3['Bearer Id'].fillna(df3['Bearer Id'].mode()[0])
df3['IMSI'] = df3['IMSI'].fillna(df3['IMSI'].mode()[0])
df3['Handset Manufacturer'] = df3['Handset Manufacturer'].fillna(df3['Handset Manufacturer'].mode()[0])

In [10]:
#filling missing average RTT DL and UL values with the median value
df3['Avg RTT DL (ms)'] = df3['Avg RTT DL (ms)'].fillna(df3['Avg RTT DL (ms)'].median())
df3['Avg RTT UL (ms)'] = df3['Avg RTT UL (ms)'].fillna(df3['Avg RTT UL (ms)'].median())

In [11]:
calculateMissing(df3)

                         variables  count  percent
0                        Bearer Id      0      0.0
1                            Start      1      0.0
2                         Start ms      1      0.0
3                              End      1      0.0
4                           End ms      1      0.0
5                        Dur. (ms)      1      0.0
6                             IMSI      0      0.0
7                    MSISDN/Number   1066      1.0
8                             IMEI    572      0.0
9               Last Location Name   1153      1.0
10                 Avg RTT DL (ms)      0      0.0
11                 Avg RTT UL (ms)      0      0.0
12         Avg Bearer TP DL (kbps)      1      0.0
13         Avg Bearer TP UL (kbps)      1      0.0
14             DL TP < 50 Kbps (%)    754      1.0
15  50 Kbps < DL TP < 250 Kbps (%)    754      1.0
16   250 Kbps < DL TP < 1 Mbps (%)    754      1.0
17              DL TP > 1 Mbps (%)    754      1.0
18             UL TP < 10 Kbps 

In [12]:
# droping all other rows with missing values. becuase they are equal to or less than 1%.
df3.dropna(inplace=True)
calculateMissing(df3)

                         variables  count  percent
0                        Bearer Id      0      0.0
1                            Start      0      0.0
2                         Start ms      0      0.0
3                              End      0      0.0
4                           End ms      0      0.0
5                        Dur. (ms)      0      0.0
6                             IMSI      0      0.0
7                    MSISDN/Number      0      0.0
8                             IMEI      0      0.0
9               Last Location Name      0      0.0
10                 Avg RTT DL (ms)      0      0.0
11                 Avg RTT UL (ms)      0      0.0
12         Avg Bearer TP DL (kbps)      0      0.0
13         Avg Bearer TP UL (kbps)      0      0.0
14             DL TP < 50 Kbps (%)      0      0.0
15  50 Kbps < DL TP < 250 Kbps (%)      0      0.0
16   250 Kbps < DL TP < 1 Mbps (%)      0      0.0
17              DL TP > 1 Mbps (%)      0      0.0
18             UL TP < 10 Kbps 

In [14]:
# counting unique users
print(len(pd.unique(df3['MSISDN/Number'])))
print(df3.shape)

105719
(146891, 45)


In [15]:
# removing duplicate users
df3 = df3.drop_duplicates(subset='MSISDN/Number', keep='first')
print(df3.shape)

(105719, 45)


<h1>Querying using sql</h1>

In [21]:
# creating a querying function.
queryDf = lambda q: sqldf(q, globals())


In [36]:
def getTopHandsets(num):
    query = 'SELECT "Handset Type", count(*) as user_count FROM df3 WHERE "Handset Type" != "undefined" group by "Handset Type" order by user_count DESC LIMIT '+str(num)
    return queryDf(query)

print(getTopHandsets(10))

                   Handset Type  user_count
0              Huawei B528S-23A       10492
1       Apple iPhone 6S (A1688)        6731
2        Apple iPhone 6 (A1586)        6224
3        Apple iPhone 7 (A1778)        4661
4       Apple iPhone Se (A1723)        3731
5        Apple iPhone 8 (A1905)        3528
6  Samsung Galaxy S8 (Sm-G950F)        3215
7       Apple iPhone Xr (A2105)        3058
8   Samsung Galaxy J5 (Sm-J530)        2733
9    Samsung Galaxy A5 Sm-A520F        2689


In [37]:
def getTopManu(num):
    query = 'SELECT "Handset Manufacturer", count(*) as user_count FROM df3 WHERE "Handset Type" != "undefined" group by "Handset Manufacturer" order by user_count DESC LIMIT '+str(num)
    return queryDf(query)
print(getTopManu(3))

  Handset Manufacturer  user_count
0                Apple       42320
1              Samsung       30436
2               Huawei       21411


In [38]:
def manByHandset(lis, dfname):
    for man in lis:
        query = 'SELECT "Handset Manufacturer", "Handset Type", count(*) as num_users \
        FROM '+dfname+'\
        WHERE "Handset Manufacturer" = "'+man+'" \
        group by "Handset Type" \
        order by num_users DESC \
        LIMIT 3'
        print(queryDf(query),'\n')

manByHandset(getTopManu(5)["Handset Manufacturer"].to_list(),'df3')

  Handset Manufacturer             Handset Type  num_users
0                Apple  Apple iPhone 6S (A1688)       6731
1                Apple   Apple iPhone 6 (A1586)       6224
2                Apple   Apple iPhone 7 (A1778)       4661 

  Handset Manufacturer                  Handset Type  num_users
0              Samsung  Samsung Galaxy S8 (Sm-G950F)       3215
1              Samsung   Samsung Galaxy J5 (Sm-J530)       2733
2              Samsung    Samsung Galaxy A5 Sm-A520F       2689 

  Handset Manufacturer                    Handset Type  num_users
0               Huawei                Huawei B528S-23A      10492
1               Huawei  Huawei P20 Lite Huawei Nova 3E       1515
2               Huawei                    Huawei E5180       1181 

            Handset Manufacturer                        Handset Type  \
0  Sony Mobile Communications Ab        Sony Mobile Comm. Xperia Xa2   
1  Sony Mobile Communications Ab  Sony Mobile Comm. Xperia Xa1 Ultra   
2  Sony Mobile Communi